In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from sklearn.model_selection import train_test_split
from textblob import TextBlob, Word
from wordcloud import WordCloud,STOPWORDS
from subprocess import check_output
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
df=pd.read_csv("./Sentiment.csv")
df.head()
df['sentiment'].value_counts()

sentiment
Negative    8493
Neutral     3142
Positive    2236
Name: count, dtype: int64

In [29]:
# indices_to_drop = df[df['sentiment'] == 'Negative'].index[1::5]

# # Drop rows with the obtained indices
# df.drop(indices_to_drop, inplace=True)

# # Reset the index after dropping rows
# df.reset_index(drop=True, inplace=True)


In [30]:
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [31]:
df['sentiment'].value_counts()

sentiment
Negative    8493
Neutral     3142
Positive    2236
Name: count, dtype: int64

In [32]:
df.shape

(13871, 21)

In [33]:
df=df[["text","sentiment"]]
df.shape

(13871, 2)

In [34]:
df.sample(5)

,text,sentiment
331,RT @BBCJonSopel: In midst of #GOPDebate came b...,Neutral
7289,RT @tarynoneill: THERE WAS NOT ONE QUESTION ON...,Negative
10450,"They were lynching Black folks in ""Jesus Name""...",Negative
11209,RT @RWSurferGirl: Fox is cherry picking the ca...,Negative
1454,Marco Rubio debating like a champ. #partyofthe...,Positive


In [35]:
# df = df[df.sentiment != "Neutral"]
# df['sentiment']= pd.get_dummies(df['sentiment'], drop_first = True)
# df = df.reset_index(drop=True)
# df.head()

In [36]:
def cleantxt(text):
    text= re.sub(r'@[A-Za-z0-9]+', '',text)# removed @mentions
    text= re.sub(r'#', '',text)# removed # symbol
    text = re.sub(r'RT[\s]+', '',text)# rmoved RT
    text = re.sub(r'https?:\/\/\s+', '',text)# removed the hyperlink
    text = re.sub(r':+', '',text)# removed : symbol
    text = re.sub(r'--+', '',text)# removed : symbol
    text = re.sub(r'http', '',text)
    return text
df["text"] = df["text"].apply(cleantxt)
df["text"] = df["text"].apply(lambda el: el.replace("RT ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("GOPDebate ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("GOPDebates ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("GOPdebate ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("gopdebate ", ""))
df["text"] = df["text"].str.replace("@.+: ","")
df["text"] = df["text"].apply(lambda el: el.replace("w/ ", ""))
df["text"] = df["text"].apply(lambda el: el.replace("TedCruz ", "Ted Cruz "))
df["text"] = df["text"].apply(lambda el: el.replace("@realDonaldTrump ", "Donald Trump "))
df["text"] = df["text"].str.replace(".+:\/.+t.co\/.+", "")
df["text"] = df["text"].str.replace("(#.+ )|(#.+)", "")
df["text"] = df["text"].str.replace("@.+ ", "")
# df["text"] = df["text"].apply(lambda el: replace_emoj(el))
df["text"] = df["text"].str.replace("[^\w\s]","")
df["text"] = df["text"].apply(lambda el: el.strip())
df["text"] = df["text"].apply(lambda el: " ".join(el for el in el.split() if el not in nltk.corpus.stopwords.words("english")))
# df["text"] = df["text"].apply(lambda el: " ".join([str(TextBlob(e).correct()) for e in el.split()]))
df["text"] = df["text"].apply(lambda el: " ".join([Word(e).lemmatize() for e in el.split()]))
df.sample(10)

,text,sentiment
4927,"What point ding-bat ""commentary"" GOPDebate?",Negative
13427,"While Ted Cruz speaking, America parent tryin'...",Negative
5334,Vets politician's afterthoughts. Squeaky wheel...,Neutral
12187,The candidate attack Fox stoping speaking. 🇺🇸 ...,Negative
12463,Jed bush win presidency mean get hilarious sho...,Negative
8282,Two great debates! Who favorites? Let know tho...,Positive
11660,"Same mistake Republicans always made, still ma...",Negative
11188,"Why chant ""war"" together? GOPdebates republica...",Negative
10433,_f Does God super PAC? GOPDebates,Neutral
10855,Finally question Cruz. Iranian deal terrible R...,Negative


In [37]:
# # ///////////////////////////////////////////////////////////////////
# from nrclex import NRCLex
# # df=df["text"]
# df['emotion']=df['text'].apply(lambda x: NRCLex(x).affect_frequencies)
# df=pd.concat([df.drop(['emotion'],axis=1),df['emotion'].apply(pd.Series)],axis=1)

In [38]:
# df

In [39]:
X=df['text'].values
y=df['sentiment'].values

In [40]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=50)
print(X_test)

['Breaking Brian Williams handed Chris Christie donut calm down. 🇺🇸 GOPDebates'
 "Most air time last night THE FOX MODERATORS GOPDebate Really? Why would make see fake ? That's nasty."
 'Jeb Bush Im prolife others Schiavo. GOPdebates' ...
 'slaying //t.co/sLFVbkScmS'
 "Not Carson disqualified, I'm sure I'd go I brain tumor GOPDebates"
 'Reps r like minions, searching 4 guy act evil 2 make boss. could learn movie.']


In [41]:
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [42]:
model=LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [43]:
#accuracy score on the training data
X_train_pred=model.predict(X_train)
train_data_acc=accuracy_score(y_train,X_train_pred)
print("Training Data Accuracy",train_data_acc)

Training Data Accuracy 0.7947909156452776


In [44]:
#accuracy score on the testing data
X_test_pred=model.predict(X_test)
test_data_acc=accuracy_score(y_test,X_test_pred)

In [45]:
print("Testing Data Accuracy",test_data_acc)

Testing Data Accuracy 0.6965765765765766


In [46]:
print(classification_report(y_test, X_test_pred))
print(y_test)

              precision    recall  f1-score   support

    Negative       0.73      0.90      0.80      1719
     Neutral       0.54      0.32      0.40       611
    Positive       0.67      0.42      0.51       445

    accuracy                           0.70      2775
   macro avg       0.65      0.55      0.57      2775
weighted avg       0.68      0.70      0.67      2775

['Negative' 'Negative' 'Neutral' ... 'Positive' 'Negative' 'Negative']


In [47]:
positive_mask = (y_test =='Positive')
negative_mask = (y_test == 'Negative')
neutral_mask = (y_test == 'Neutral')

# Calculate accuracy for each sentiment category
accuracy_positive = accuracy_score(y_test[positive_mask], X_test_pred[positive_mask])
accuracy_negative = accuracy_score(y_test[negative_mask], X_test_pred[negative_mask])
accuracy_neutral = accuracy_score(y_test[neutral_mask], X_test_pred[neutral_mask])

print("Accuracy for Positive Sentiments:", accuracy_positive)
print("Accuracy for Negative Sentiments:", accuracy_negative)
print("Accuracy for Neutral Sentiments:", accuracy_neutral)

Accuracy for Positive Sentiments: 0.41797752808988764
Accuracy for Negative Sentiments: 0.9028504944735312
Accuracy for Neutral Sentiments: 0.3191489361702128


In [48]:
# # here i am creating function to get subjectivity
# def getSubjectivity(text):
#     return TextBlob(text).sentiment.subjectivity
# # here i am creating function to get polarity
# def getPolarity(text):
#     return TextBlob(text).sentiment.polarity
# # here i am creating two new column of subjectivity and polarity
# df["subjectivity"] = df["text"].apply(getSubjectivity)
# df["polarity"] = df["text"].apply(getPolarity)

In [49]:
# allwords = ' '.join([twts for twts in df["text"]])
# wordcloud = WordCloud(width=2500,
#                       height=2000,stopwords=STOPWORDS,background_color="white",random_state=21
#                      ).generate(allwords)
# plt.figure(1,figsize=(10,10))
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.show()

In [50]:
# #training set after doing visualisation
# text = []
# stopwords_set = set(stopwords.words("english"))

# for index, row in df.iterrows():
#     words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
#     words_cleaned = [word for word in words_filtered
#         if 'http' not in word
#         and not word.startswith('@')
#         and not word.startswith('#')
#         and word != 'RT']
#     words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
#     text.append((words_without_stopwords, row.sentiment))

In [51]:
# df.sample(5)

In [52]:
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=50)
# print(X_test)
# vectorizer=TfidfVectorizer()
# X_train=vectorizer.fit_transform(X_train)
# X_test=vectorizer.transform(X_test)
# model=LogisticRegression(max_iter=1000)
# model.fit(X_train,y_train)
# #accuracy score on the training data
# X_train_pred=model.predict(X_train)
# train_data_acc=accuracy_score(y_train,X_train_pred)
# print("Training Data Accuracy",train_data_acc)
# #accuracy score on the testing data
# X_test_pred=model.predict(X_test)
# test_data_acc=accuracy_score(y_test,X_test_pred)
# print("Testing Data Accuracy",test_data_acc)